C:\Users


In [53]:



import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
from astropy.io import fits
from astropy.utils.data import download_file
from matplotlib.colors import LogNorm

x = './trab/desktop/xo2b'

def bias(y):
    a = glob.glob(x+"/bias/*.fits")
    masterbias = []
    for i in range(len(a)):
        img = fits.getdata(a[i])
        img = img.astype(np.float64)
        masterbias.append(img)
    if y == 1:
        return np.median(masterbias,axis=0)
    else:
        outfile = x + '/master_bias.fits'
        hdu = fits.PrimaryHDU() #criando o HDU 
        hdu.data = np.median(masterbias,axis=0)
        hdu.writeto(outfile)

In [54]:
bias(1)


array([[287.09567261, 288.43109131, 288.09628296, ..., 317.15054321,
        314.9147644 , 314.9928894 ],
       [287.79431152, 286.50576782, 286.49328613, ..., 313.07952881,
        315.17562866, 315.28067017],
       [286.9175415 , 286.90869141, 287.59484863, ..., 316.10891724,
        314.44839478, 314.47311401],
       ...,
       [285.47119141, 286.2303772 , 286.85089111, ..., 312.75241089,
        315.2739563 , 315.29788208],
       [285.73690796, 286.86508179, 285.75009155, ..., 313.840271  ,
        314.45941162, 314.47415161],
       [285.69833374, 285.74697876, 285.37484741, ..., 313.93972778,
        313.55096436, 313.63745117]])

In [55]:
def flat(d):
    b = glob.glob(x+"/flat/*.fits")
    masterflat = []
    masterflatbias = []
    for i in range(len(b)):
        img = fits.getdata(b[i])
        img = img.astype(np.float64)
        masterflat.append(img)
        masterflatbias.append((masterflat[i] - bias(1))/np.median(masterflat[i]))
    if d == 1:
        return np.median(masterflatbias,axis=0)
    else:
        outfile = x + '/master_flat_bias.fits'
        hdu = fits.PrimaryHDU() #criando o HDU 
        hdu.data = np.median(masterflatbias,axis=0)
        hdu.writeto(outfile)

In [56]:
flat(1)

array([[1.08630775, 0.31210575, 0.31779009, ..., 0.33572358, 1.25562973,
        1.25602728],
       [0.90544953, 0.22328618, 0.22743632, ..., 0.22414036, 1.1093883 ,
        1.10972952],
       [0.92033337, 0.25336732, 0.25413839, ..., 0.25696047, 1.19559382,
        1.19594568],
       ...,
       [0.97763469, 0.26038239, 0.26106341, ..., 0.26731792, 1.17567192,
        1.17594639],
       [0.9544005 , 0.26088381, 0.26123909, ..., 0.27186694, 1.15585305,
        1.15611839],
       [0.90459092, 0.26285855, 0.26133482, ..., 0.2685868 , 1.07266922,
        1.07291897]])

In [59]:
def science():
    c = glob.glob(x+'/science/*.fits')
    for i in range(len(c)):
        img = fits.getdata(c[i])
        img = img.astype(np.float64)
        sci_clean = (img-bias(1))/flat(1)
        outfile = c[i][:-5] + "_clean.fits"
        hdu = fits.PrimaryHDU()
        hdu.data = sci_clean
        hdu.writeto(outfile)
        
        

'./trab/desktop/xo2b/science\\xo2b.0004.fits'

'./trab/desktop/xo2b/science\\xo2b.0004'